In [1]:
#Working on applying correlations to the data

#Importing the necessary libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from prophet import Prophet
import scipy.stats as stats
import pickle


In [2]:
#Read in data

df_utahpop = pd.read_csv('../Usefull Dataframes/df_utahpop.csv')
df_rmhi = pd.read_csv('../Resources/Real Median Household Income.csv')
mhpu_df = pd.read_csv('../Usefull Dataframes/Median_Home_Prices_Monthly_In_Thousands.csv')

with open('../Pop vs Build Permits/Resources/permits_df.pkl', 'rb') as f:
    permits_df = pickle.load(f)



    

In [3]:
display(permits_df.head())
display(df_rmhi.head())
display(df_utahpop.head())
display(mhpu_df.head())

,DATE,UTBP1FH,Month on Month Change,Percent Change,Total Permits
144,2000-01-01,944.0,NaN,NaN,944.0
145,2000-02-01,1082.0,138.0,0.146186,2026.0
146,2000-03-01,1312.0,230.0,0.212569,3338.0
147,2000-04-01,1343.0,31.0,0.023628,4681.0
148,2000-05-01,1363.0,20.0,0.014892,6044.0


,DATE,MEHOINUSUTA672N
0,1984-01-01,58410
1,1985-01-01,61830
2,1986-01-01,63350
3,1987-01-01,61850
4,1988-01-01,59160


,Year,Population,Year on Year Change,Change in Percent
0,2001-01-01,2282415,38208,1.70%
1,2002-01-01,2322002,39587,1.73%
2,2003-01-01,2355755,33753,1.45%
3,2004-01-01,2395448,39693,1.68%
4,2005-01-01,2449105,53657,2.24%


,Date,Home Prices
0,2000-01-01,164.74
1,2000-02-01,164.83
2,2000-03-01,164.97
3,2000-04-01,165.33
4,2000-05-01,166.62


In [5]:
#Clean data and prepare them for merge and analysis

#Start with interpolating the annual population year on year change to be a month to month change

#change the 'Year' column to 'Date'
df_utahpop.rename(columns={'Year':'Date'}, inplace=True)

df_utahpop['Date'] = pd.to_datetime(df_utahpop['Date'])

#Interpolate the rows to have monthly instead of annual columns
df_utahpop = df_utahpop.resample('MS', on='Date').first().reset_index()


df_utahpop_yoy = df_utahpop[['Date','Year on Year Change']]

#interpolate the values between each yearly datapoint
df_utahpop_yoy = df_utahpop_yoy.interpolate(method='linear')

df_utahpop_yoy.rename(columns={'Year on Year Change':'Month on Month Change'}, inplace=True)

df_utahpop_yoy.head(25)





,Date,Month on Month Change
0,2001-01-01,38208.000000
1,2001-02-01,38322.916667
2,2001-03-01,38437.833333
3,2001-04-01,38552.750000
4,2001-05-01,38667.666667
5,2001-06-01,38782.583333
6,2001-07-01,38897.500000
7,2001-08-01,39012.416667
8,2001-09-01,39127.333333
9,2001-10-01,39242.250000


In [6]:
#rename the 'DATE' column to 'Date' to match the other dataframes
df_rmhi.rename(columns={'DATE':'Date'}, inplace=True)

#change the 'Date' column to datetime

df_rmhi['Date'] = pd.to_datetime(df_rmhi['Date'])

#Change the 'MEHOIUSUTA672N' column to 'Real Median Household Income'
df_rmhi.rename(columns={'MEHOINUSUTA672N':'Real Median Household Income'}, inplace=True)

#Change the units in the 'Real Median Household Income' column to have a dollar sign and have a comma for the thousands
df_rmhi['Real Median Household Income'] = df_rmhi['Real Median Household Income'].map('${:,.0f}'.format)


df_rmhi.head()

,Date,Real Median Household Income
0,1984-01-01,"$58,410"
1,1985-01-01,"$61,830"
2,1986-01-01,"$63,350"
3,1987-01-01,"$61,850"
4,1988-01-01,"$59,160"


In [18]:
# Slice mhpu_df to only include dates starting from 2001-01-01
mhpu_df = mhpu_df[mhpu_df['Date'] >= '2001-01-01']

#convert the 'Date' column to datetime
mhpu_df['Date'] = pd.to_datetime(mhpu_df['Date'])

mhpu_df.head()


,Date,Home Prices
12,2001-01-01,172.53
13,2001-02-01,172.80
14,2001-03-01,172.97
15,2001-04-01,173.14
16,2001-05-01,173.24


In [22]:
#Change the 'DATE column to 'Date'
permits_df.rename(columns={'DATE':'Date'}, inplace=True)

# Slice permits_df to only include dates starting from 2001-01-01
permits_df = permits_df[permits_df['Date'] >= '2001-01-01']

#convert the 'Date' column to datetime
permits_df['Date'] = pd.to_datetime(permits_df['Date'])

cln_permits_df = permits_df[['Date','Total Permits']]

cln_permits_df.head()



,Date,Total Permits
156,2001-01-01,14671.0
157,2001-02-01,15632.0
158,2001-03-01,16983.0
159,2001-04-01,18191.0
160,2001-05-01,19586.0


In [ ]:
# Merge the dataframes on the 'Date' column
merged_df = pd.merge(df_utahpop_yoy, mhpu_df, on='Date')

merged_df.tail()